# Introduction
Aim of the project is to predict wheter a molecule elicit a biological response or not. The project first appeared as Kaggle [Bioresponse Competition](https://www.kaggle.com/c/bioresponse).
It is a Binary Classification problem; the target variable is `Activity`: an experimentally measured biological response to molecules in the dataset. `Activity` is `1` if there is a measured biological response, otherwise it is `0`.
Available features are 1776 Molecular Descriptors (columns `D1` - `D1776`). 

The "physical meaning" of the available Molecular Descriptors, as well as the specific Biological Response which was measured, are not made known to competition partecipants. No "domain knowledge" is applicable.

# Library Imports

In [ ]:
from time import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss

# Exploratory Analysis

In [ ]:
df = pd.read_csv("input/bioresponse/train.csv")
#df = df.sample(100, random_state=1)
df.head()

In [ ]:
df.shape

No missing values:

In [ ]:
df.isnull().any().any()

In [ ]:
df.describe()

In [ ]:
target_name = "Activity"
y = df.loc[:, target_name]
X = df.drop(target_name, axis="columns")

Dataset is balanced:

In [ ]:
y.value_counts()

In [ ]:
y.mean()

There are many input features, so it is pointless to visualize each one.
We are informed by [Data Description](https://www.kaggle.com/c/bioresponse/data) that input features are already normalized. Can we visually confirm that information, by looking at Violin Plots of Maximum, Minimum and Mean of Features? What about Variance?

In [ ]:
stat_functions = ["min", "mean", "max", "var"]

stats = X.apply(stat_functions).T
stats["Descriptor"] = stats.index
stats.reset_index(inplace=True, drop=True)
stats.head()

In [ ]:
for fun in stat_functions:
    sns.violinplot(data=stats, x=fun)
    plt.show()

Minimum is peaked around 0, but some values are much higher.
Maximum is peaked around 1, but some values are much lower.
Mean distribution is simular to Minimum distribution, but the peak is less pronounced.


It is confirmed that Normalization, i.e. "Min-Max Scaling", was applied to the Descriptors. But predictive models may still benefit from other preliminary scaling techniques.

Variance is around 0 for most of features, but some values are higher, up to 0.3 .

Are Minimum, Maximum, Mean, and Variance, of Descriptors equally distributed between active and inactive compounds?

In [ ]:
X_active = df.query(f"{target_name} == 1").drop(target_name, axis="columns")
X_inactive = df.query(f"{target_name} == 0").drop(target_name, axis="columns")

stats_active = X_active.apply(stat_functions).T
stats_active[target_name] = "active"

stats_inactive = X_inactive.apply(stat_functions).T
stats_inactive[target_name] = "inactive"

stats = pd.concat([stats_active, stats_inactive])
stats["Descriptor"] = stats.index
stats.reset_index(inplace=True, drop=True)
stats.head()

In [ ]:
for fun in stat_functions:
    sns.violinplot(data=stats, x=fun, y=target_name)
    plt.show()

We can confirm that Descriptor statistics are equally distributed between active and inactive compounds.


Are the Descriptors correlated?

In [ ]:
corr = X.corr()

mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True

fig, ax = plt.subplots(constrained_layout=True, figsize=(10, 10))
sns.heatmap(corr,
            vmin=-1,
            vmax=1,
            cmap="PRGn",
            mask=mask,
            square=True,
            ax=ax,
            );

Some features are positively correlated. A lower number of features are negatively correlated. Most features have small correlation. Still, the dataset may benefit from some kind of dimensionality reduction, such as PCA.

Try PCA without scaling:

In [ ]:
pca = PCA()
pcs = pca.fit_transform(X)

pc_names = [f"PC{i}" for i in range(1, pcs.shape[1] + 1)]

pcs = pd.DataFrame(pcs, columns=pc_names)
pcs[target_name] = y.astype("category")

expl_ratios = pca.explained_variance_ratio_
expl_ratios = pd.Series(expl_ratios, index=pc_names)

In [ ]:
pc_x = 1
pc_y = 2
expl_ratio_x = expl_ratios.at[f"PC{pc_x}"]
expl_ratio_y = expl_ratios.at[f"PC{pc_y}"]

sns.lmplot(data=pcs,
           x=f"PC{pc_x}",
           y=f"PC{pc_y}",
           hue=target_name,
           fit_reg=False,
           scatter_kws={"alpha": 0.1},
           )

ax = plt.gca()
ax.set_xlabel(f"PC{pc_x}: {expl_ratio_x * 100:0.1f}%")
ax.set_ylabel(f"PC{pc_y}: {expl_ratio_y * 100:0.1f}%");

Three clusters are visible using PC1 and PC2. They do not separe active from inactive compounds.

In [ ]:
cumsum_expl_ratios = expl_ratios.cumsum()
cumsum_expl_ratios.plot();

In [ ]:
tresholds_cumsum_expl_ratios = [0.7, 0.8, 0.9]


n_pcs_for_tresholds_unscaled = [(cumsum_expl_ratios <= tresh).argmin() + 1
                                for tresh in tresholds_cumsum_expl_ratios]

for tresh, n_pcs in zip(tresholds_cumsum_expl_ratios, n_pcs_for_tresholds_unscaled):
    print(f"{n_pcs} components are needed to have {tresh * 100}% variance")

So, without scaling, 242 components are needed to have 90% variance!

Try PCA with scaling:

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


pca = PCA()
pcs = pca.fit_transform(X_scaled)

pc_names = [f"PC{i}" for i in range(1, pcs.shape[1] + 1)]

pcs = pd.DataFrame(pcs, columns=pc_names)
pcs[target_name] = y.astype("category")

expl_ratios = pca.explained_variance_ratio_
expl_ratios = pd.Series(expl_ratios, index=pc_names)

In [ ]:
pc_x = 1
pc_y = 2
expl_ratio_x = expl_ratios.at[f"PC{pc_x}"]
expl_ratio_y = expl_ratios.at[f"PC{pc_y}"]

sns.lmplot(data=pcs,
           x=f"PC{pc_x}",
           y=f"PC{pc_y}",
           hue=target_name,
           fit_reg=False,
           scatter_kws={"alpha": 0.1},
           )

ax = plt.gca()
ax.set_xlabel(f"PC{pc_x}: {expl_ratio_x * 100:0.1f}%")
ax.set_ylabel(f"PC{pc_y}: {expl_ratio_y * 100:0.1f}%");

Two clusters are visible when plotting PC1 vs PC2, but they do not separe active from inactive compounds.

In [ ]:
cumsum_expl_ratios = expl_ratios.cumsum()
cumsum_expl_ratios.plot();

In [ ]:
n_pcs_for_tresholds_scaled = [(cumsum_expl_ratios <= tresh).argmin() + 1
                              for tresh in tresholds_cumsum_expl_ratios]

for tresh, n_pcs in zip(tresholds_cumsum_expl_ratios, n_pcs_for_tresholds_scaled):
    print(f"{n_pcs} components are needed to have {tresh * 100}% variance")

So, with scaling, 350 components will produce a dataset with 90% of total variance! 350 components are less than one third of total number of original features.

# Algorithm Selection
First, create train and test datasets:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    stratify=y,
                                                    random_state=1,
                                                    )

X_train.shape, X_test.shape, y_train.shape, y_test.shape

Thanks to option `stratify=y` of `train_test_split` function, both
train and test contain a balanced amount of active and inactive compounds:

In [ ]:
y_train.mean(), y_test.mean()

Let's experiment with some models before running GridSearchCV:

In [ ]:
start_time = time()
model = make_pipeline(PCA(n_components=10), GradientBoostingClassifier())
#model = GradientBoostingClassifier()
model.fit(X_train, y_train)
end_time = time()
tot_time = end_time - start_time
tot_time / 3600

Define base hyperparameters that will be fixed in all model pipelines of each type of model:

In [ ]:
kwargs_pca_unscaled = dict(n_components=n_pcs_for_tresholds_unscaled[1])
kwargs_pca_scaled = dict(n_components=n_pcs_for_tresholds_scaled[1])
kwargs_l1 = dict(penalty="l1", solver="liblinear", random_state=1)
kwargs_l2 = dict(penalty="l2", solver="liblinear", random_state=1)
kwargs_rf = dict(random_state=1)
kwargs_gb = dict(random_state=1)

Define hyperparameter grids:

In [ ]:
#grid_pca_unscaled = {"pca__n_components": n_pcs_for_tresholds_unscaled}
#grid_pca_scaled = {"pca__n_components": n_pcs_for_tresholds_scaled}
grid_l1 = {"logisticregression__C": [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100, 500, 1000]}
grid_l2 = grid_l1
grid_rf = {"randomforestclassifier__n_estimators": [100, 200],
           "randomforestclassifier__max_features": ["auto", "sqrt", 0.33],
           "randomforestclassifier__min_samples_leaf": [1, 2, 3, 5, 10],
           }
grid_gb = {"gradientboostingclassifier__n_estimators": [100, 200],
           "gradientboostingclassifier__learning_rate": [0.03, 0.05, 0.1, 0.2],
           "gradientboostingclassifier__max_depth": [1, 2, 3, 5, None],
           }

hyperparameters = {"l1": grid_l1,
                   "l2": grid_l2,
                   "rf": grid_rf,
                   "gb": grid_gb,
                   }

Define model pipelines: for each model type, different kinds of preprocessing will be tried. The model will be applied on a number of PCS corresponding to the previously defined tresholds, and PCA will be applied on the dataset as-is and on the scaled dataset

First, the models without scaling will be fitted:

In [ ]:
pipelines_unscaled = {"l1": make_pipeline(PCA(**kwargs_pca_unscaled),
                                          LogisticRegression(**kwargs_l1)),
                      "l2": make_pipeline(PCA(**kwargs_pca_unscaled),
                                          LogisticRegression(**kwargs_l2)),
                      "rf": make_pipeline(PCA(**kwargs_pca_unscaled),
                                          RandomForestClassifier(**kwargs_rf)),
                      "gb": make_pipeline(PCA(**kwargs_pca_unscaled),
                                          GradientBoostingClassifier(**kwargs_gb)),
                      }

In [ ]:
fitted_models_unscaled = {}
model_short_names = ["l1", "l2", "gb", "rf"]
#model_short_names = ["l1", "l2"]
for name in model_short_names:
    start_time = time()
    
    model = GridSearchCV(estimator=pipelines_unscaled.get(name),
                         param_grid=hyperparameters.get(name),
                         n_jobs=5,
                         )
    
    model.fit(X_train, y_train),
    
    end_time = time()
    tot_time = end_time - start_time
    print(f"{name} has been fitted in {tot_time / 3600:0.6f} hours!")
    
    fitted_models_unscaled.update({name: model})

In [ ]:
df_results_unscaled = []
for name, model in fitted_models_unscaled.items():
    
    best_score = model.best_score_
    
    pred_proba = model.predict_proba(X_test)[:, 1]
    
    auroc = roc_auc_score(y_test, pred_proba)
    loss = log_loss(y_test, pred_proba)
    
    tmp = {"model": name,
           "accuracy_train": best_score,
           "auroc_test": auroc,
           "log_loss_test": loss,
           }
    df_results_unscaled.append(tmp)

df_results_unscaled = pd.DataFrame(df_results_unscaled)
for score_name in ["accuracy_train", "auroc_test", "log_loss_test"]:
    df_results_unscaled[f"rank_{score_name}"] = df_results_unscaled[score_name].rank(ascending=False).astype(int)

df_results_unscaled